# Pandas de alto rendimiento: eval() y query()

Como ya hemos visto en notebook anteriores, la potencia de PyData se basa en la capacidad de NumPy y Pandas para introducir operaciones básicas en C a través de una sintaxis intuitiva: algunos ejemplos son las operaciones vectorizadas/transmitidas en NumPy, y las operaciones de tipo agrupación en Pandas.
Aunque estas abstracciones son eficientes y eficaces para muchos casos de uso común, a menudo dependen de la creación de objetos intermedios temporales, que pueden causar una sobrecarga indebida en el tiempo de cálculo y el uso de memoria.

Pandas incluye algunas herramientas que permiten acceder directamente a operaciones de velocidad C sin la costosa asignación de arrays intermedios.
Se trata de las funciones ``eval()`` y ``query()``.
En este cuaderno repasaremos su uso y daremos algunas reglas sobre cuándo deberías pensar en usarlas.

## Motivación de ``query()`` y ``eval()``: Expresiones compuestas

Hemos visto anteriormente que NumPy y Pandas soportan operaciones vectoriales rápidas; por ejemplo, al sumar los elementos de dos arrays:

In [ ]:
import numpy as np
rng = np.random.RandomState(42)
x = rng.rand(1000000)
y = rng.rand(1000000)
%timeit x + y

Esto es mucho más rápido que hacer la suma mediante un bucle o comprensión de Python:

In [ ]:
%timeit np.fromiter((xi + yi for xi, yi in zip(x, y)), dtype=x.dtype, count=len(x))

Pero esta abstracción puede perder eficacia al calcular expresiones compuestas.
Por ejemplo, considere la siguiente expresión:

In [ ]:
mask = (x > 0.5) & (y < 0.5)

Dado que NumPy evalúa cada subexpresión, esto equivale aproximadamente a lo siguiente:

In [ ]:
tmp1 = (x > 0.5)
tmp2 = (y < 0.5)
mask = tmp1 & tmp2

El beneficio aquí es que evalúa la expresión de una manera que no utiliza arrays temporales de tamaño completo, y por lo tanto puede ser mucho más eficiente que NumPy, especialmente para arrays grandes.

## ``pandas.eval()`` para operaciones eficientes

La función ``eval()`` de Pandas utiliza expresiones de cadena para calcular eficientemente operaciones utilizando ``DataFrame``.
Por ejemplo, considera los siguientes ``DataFrame``:

In [ ]:
import pandas as pd
nrows, ncols = 100000, 100
rng = np.random.RandomState(42)
df1, df2, df3, df4 = (pd.DataFrame(rng.rand(nrows, ncols))
                      for i in range(4))

Para calcular la suma de los cuatro ``DataFrame`` utilizando el enfoque típico de Pandas, podemos simplemente escribir la suma:

In [ ]:
%timeit df1 + df2 + df3 + df4

El mismo resultado puede calcularse mediante ``pd.eval`` construyendo la expresión como una string:

In [ ]:
%timeit pd.eval('df1 + df2 + df3 + df4')

La versión ``eval()`` de esta expresión es aproximadamente un 50% más rápida (y utiliza mucha menos memoria), pero da el mismo resultado:

In [ ]:
np.allclose(df1 + df2 + df3 + df4,
            pd.eval('df1 + df2 + df3 + df4'))

### Operaciones soportadas por ``pd.eval()``

Pandas ``pd.eval()`` soporta un amplio rango de operaciones.
Para demostrarlas, usaremos los siguientes ``DataFrame`` enteros:

In [ ]:
df1, df2, df3, df4, df5 = (pd.DataFrame(rng.randint(0, 1000, (100, 3)))
                           for i in range(5))

#### Operadores aritméticos
``pd.eval()`` soporta todos los operadores aritméticos. Por ejemplo:

In [ ]:
result1 = -df1 * df2 / (df3 + df4) - df5
result2 = pd.eval('-df1 * df2 / (df3 + df4) - df5')
np.allclose(result1, result2)

#### Operadores de comparación
``pd.eval()`` soporta todos los operadores de comparación, incluidas las expresiones encadenadas:

In [ ]:
result1 = (df1 < df2) & (df2 <= df3) & (df3 != df4)
result2 = pd.eval('df1 < df2 <= df3 != df4')
np.allclose(result1, result2)

#### Operadores bit a bit
``pd.eval()`` soporta los operadores bit a bit ``&`` y ``|``:

In [ ]:
result1 = (df1 < 0.5) & (df2 < 0.5) | (df3 < df4)
result2 = pd.eval('(df1 < 0.5) & (df2 < 0.5) | (df3 < df4)')
np.allclose(result1, result2)

Además, admite el uso de los literales ``and`` y ``or`` en expresiones booleanas:

In [ ]:
result3 = pd.eval('(df1 < 0.5) and (df2 < 0.5) or (df3 < df4)')
np.allclose(result1, result3)

#### Atributos e índices de objetos

``pd.eval()`` permite acceder a los atributos de los objetos mediante la sintaxis ``obj.attr``, y a los índices mediante la sintaxis ``obj[index]``:

In [ ]:
result1 = df2.T[0] + df3.iloc[1]
result2 = pd.eval('df2.T[0] + df3.iloc[1]')
np.allclose(result1, result2)

## ``DataFrame.eval()`` para operaciones por columnas

Al igual que Pandas tiene una función de alto nivel ``pd.eval()``, ``DataFrame`` tiene un método ``eval()`` que funciona de manera similar.
La ventaja del método ``eval()`` es que se puede hacer referencia a las columnas *por su nombre*.
Usaremos este array etiquetado como ejemplo:

In [ ]:
df = pd.DataFrame(rng.rand(1000, 3), columns=['A', 'B', 'C'])
df.head()

Usando ``pd.eval()`` como arriba, podemos calcular expresiones con las tres columnas así:

In [ ]:
result1 = (df['A'] + df['B']) / (df['C'] - 1)
result2 = pd.eval("(df.A + df.B) / (df.C - 1)")
np.allclose(result1, result2)

El método ``DataFrame.eval()`` permite una evaluación mucho más sucinta de las expresiones con las columnas:

In [ ]:
result3 = df.eval('(A + B) / (C - 1)')
np.allclose(result1, result3)

Observe que aquí tratamos los *nombres de columna como variables* dentro de la expresión evaluada, y el resultado es el que desearíamos.

### Asignación en DataFrame.eval()

Además de las opciones que acabamos de discutir, ``DataFrame.eval()`` también permite la asignación a cualquier columna.
Usemos el ``DataFrame`` de antes, que tiene las columnas ``'A'``, ``'B'``, y ``'C'``:

In [ ]:
df.head()

Podemos utilizar ``df.eval()`` para crear una nueva columna ``'D'`` y asignarle un valor calculado a partir de las otras columnas:

In [ ]:
df.eval('D = (A + B) / C', inplace=True)
df.head()

Del mismo modo, puede modificarse cualquier columna existente:

In [ ]:
df.eval('D = (A - B) / C', inplace=True)
df.head()

### Variables locales en DataFrame.eval()

El método ``DataFrame.eval()`` soporta una sintaxis adicional que le permite trabajar con variables locales de Python.

In [ ]:
column_mean = df.mean(1)
result1 = df['A'] + column_mean
result2 = df.eval('A + @column_mean')
np.allclose(result1, result2)

El carácter ``@`` marca aquí un *nombre de variable* en lugar de un *nombre de columna*, y te permite evaluar eficientemente expresiones que involucran los dos "espacios de nombres": el espacio de nombres de las columnas, y el espacio de nombres de los objetos Python.
Ten en cuenta que este carácter ``@`` sólo está soportado por el método ``DataFrame.eval()`` *method*, no por la función ``pandas.eval()`` *function*, porque la función ``pandas.eval()`` sólo tiene acceso a un espacio de nombres (Python).

## Método DataFrame.query()

El ``DataFrame`` tiene otro método basado en cadenas evaluadas, llamado método ``query()``.
Considera lo siguiente:

In [ ]:
result1 = df[(df.A < 0.5) & (df.B < 0.5)]
result2 = pd.eval('df[(df.A < 0.5) & (df.B < 0.5)]')
np.allclose(result1, result2)

Al igual que en el ejemplo utilizado anteriormente sobre ``DataFrame.eval()``, se trata de una expresión que implica columnas del ``DataFrame``.
Sin embargo, no puede expresarse utilizando la sintaxis ``DataFrame.eval()``.
En su lugar, para este tipo de operación de filtrado, puedes utilizar el método ``query()``:

In [ ]:
result2 = df.query('A < 0.5 and B < 0.5')
np.allclose(result1, result2)

Además de ser un cálculo más eficiente, comparado con la expresión de enmascaramiento es mucho más fácil de leer y entender.
Tenga en cuenta que el método ``query()`` también acepta la bandera ``@`` para marcar variables locales:

In [ ]:
Cmean = df['C'].mean()
result1 = df[(df.A < Cmean) & (df.B < Cmean)]
result2 = df.query('A < @Cmean and B < @Cmean')
np.allclose(result1, result2)

## Rendimiento: Cuándo utilizar estas funciones

A la hora de considerar si utilizar o no estas funciones, hay dos consideraciones: *tiempo de cálculo* y *uso de memoria*.
El uso de memoria es el aspecto más predecible. Como ya se ha mencionado, cada expresión compuesta que involucre arrays de NumPy o ``DataFrame``s de Pandas resultará en la creación implícita de arrays temporales:

In [ ]:
x = df[(df.A < 0.5) & (df.B < 0.5)]

es aproximadamente equivalente a esto:

In [ ]:
tmp1 = df.A < 0.5
tmp2 = df.B < 0.5
tmp3 = tmp1 & tmp2
x = df[tmp3]

Si el tamaño del ``DataFrame`` temporal es significativo comparado con la memoria disponible de tu sistema (normalmente varios gigabytes) entonces es una buena idea usar una expresión ``eval()`` o ``query()``.
Puedes comprobar el tamaño aproximado de tu array en bytes usando esto:

In [ ]:
df.values.nbytes

Desde el punto de vista del rendimiento, ``eval()`` puede ser más rápido incluso cuando no se está maximizando la memoria del sistema.
La cuestión es cómo se comparan tus ``DataFrame`` temporales con el tamaño de la caché L1 o L2 de la CPU en tu sistema (normalmente unos pocos megabytes en 2016); si son mucho más grandes, entonces ``eval()`` puede evitar algún movimiento potencialmente lento de valores entre las diferentes cachés de memoria.
En la práctica, encuentro que la diferencia en tiempo de cálculo entre los métodos tradicionales y el método ``eval``/``query`` no suele ser significativa; en todo caso, el método tradicional es más rápido para matrices más pequeñas.
El beneficio de ``eval``/``query`` es principalmente el ahorro de memoria, y la sintaxis a veces más limpia que ofrecen.

Hemos cubierto la mayoría de los detalles de ``eval()`` y ``query()`` aquí; para más información sobre ellos, puedes consultar la documentación de Pandas.
En particular, se pueden especificar diferentes analizadores sintácticos y motores para ejecutar estas consultas; para más detalles sobre esto, vea la discusión dentro de la sección ["Enhancing Performance"](https://pandas.pydata.org/pandas-docs/dev/user_guide/enhancingperf.html).

<!--NAVIGATION-->
< [Trabajando con TimeSeries](10-Trabajar_con_TimeSeries.ipynb)
